In [18]:
import pandas as pd
import nltk
from nltk import pos_tag
nltk.download('averaged_perceptron_tagger')
from dateutil import parser
import re
from geotext import GeoText
import numpy as np
from scipy import stats 
import geopandas as gpd
import datetime

# Spacy for tokenizing our texts

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

# Gensim is needed for modeling

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from gensim.utils import tokenize

from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jrdaos\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [84]:
# Setting up Spacy Tokenizer
nlp = English()

def lemmatizer(doc):
    # This takes in a doc of tokens from the NER and lemmatizes them. 
    # Pronouns (like "I" and "you" get lemmatized to '-PRON-', so I'm removing those.
    doc = [token.lemma_ for token in doc if token.lemma_ != '-PRON-']
    doc = u' '.join(doc)
    return nlp.make_doc(doc)
    
def remove_stopwords(doc):
    # This will remove stopwords and punctuation.
    # Use token.text to return strings, which we'll need for Gensim.
    doc = [token.text for token in doc if token.is_stop != True and token.is_punct != True]
    return doc

# This will add pipelines in our tokenization process.

nlp.add_pipe(lemmatizer,name='lemmatizer')
nlp.add_pipe(remove_stopwords, name="stopwords", last=True)

In [86]:
# This is a function that will create a model that predicts the topics conveyed by each group of tweet sentiments


def topic_modeler(tokenized_texts, no_topics, no_words):
    topics = []

    words = corpora.Dictionary(tokenized_texts)
    corpus = [words.doc2bow(doc) for doc in tokenized_texts]

    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=words,
                                                random_state = 3,
                                               num_topics= no_topics)
    
    # Compute Coherence Score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_texts, dictionary=words, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()

    return lda_model

In [195]:
df = pd.read_csv('abscbn_scraped.csv')
df = df.iloc[:,1:]
df['date'] = [parser.parse(date).strftime('%Y-%m-%d') for date in df['date']]
df = df[(df['text'].str.contains('coronavirus'))]
#df = df[df['category'] == 'Philippines']
df = df.drop_duplicates()
df = df.reset_index(drop = True)



location = pd.read_csv('ph_locations.csv')
location = location.applymap(str.lower)

In [97]:
# LDA Topics 

words = df['text'].str.lower()
listWords = []
for item in words:
    listWords.append([nlp(item)])

topics = []
for x in listWords:
    res = topic_modeler(x, 1, 30)
    res = res.show_topic(0, topn = 30)
    topics.append([word[0] for word in res])
    
df['LDA_Topics'] = topics

In [197]:
# Extracting all the counting phrases in the articles

df['count_docs'] =  df['text'].apply(lambda x: re.findall("\d+(?:,\d+)?\s+[a-zA-Z]+", x))

checker = ['confirmed','suspected','quarantine','case','infected','monitoring','chinese','monitored']

count_docs = []
for index, row in df.iterrows():
    passed = []
    for item in row['count_docs']:
        if any(ext in item.lower() for ext in checker):
            passed.append(item)
            break
    
    count_docs.append(passed)

df['count_docs'] = count_docs

In [198]:
# Extracting all the PH Locations using geotext on the articles

df['PH_Loc'] = [list(set(GeoText(content, 'PH').cities)) for content in df['text']]
df['PH_Loc'] = [[x.lower() for x in w] for w in df['PH_Loc']]
df['PH_Loc'] =[[x.replace('city', '') for x in w] for w in df['PH_Loc']]

In [199]:
# Identifying which articles are about suspicious or confirmed cases of the virus

status = []
for index, row in df.iterrows():
    if ('confirmed' in row['LDA_Topics']) & ('confirm' in row['title'])  & (row['date'] >= '2020-01-30'):
        status.append('confirmed')
    elif ('confirmed' in row['LDA_Topics']) & (row['date'] >= '2020-01-30'):
        status.append('confirmed')
    elif (any(words in row['LDA_Topics']  for words in ['suspected','quarantine','case','infected','monitoring']))& ('FACT CHECK' not in row['title']) & ('FALSE' not in row['title']):
        status.append('suspected')
    else:
        status.append('')
df['status'] = status

In [200]:
# Selecting Provinces in the identified locations

df['PH_Loc'] = [list(set(loc) & set(location['Pro_Name'].unique())) for loc in df['PH_Loc']]

In [201]:
# For locations not identified through the text, it will check with the LDA topics if a location is identified and use it instead

for index, row in df.iterrows():
    if len(row['PH_Loc']) == 0:
        try:
            df.loc[index, 'PH_Loc'] = [list(set(row['LDA_Topics']) & set(location['Pro_Name'].unique()))]
        except ValueError:
            continue

In [202]:
# Cleaning the document counts to just numbers

counts = []
case = []
for count in df['count_docs']:
    try:
        counts.append(count[0].split(' ')[0])
    except IndexError:
        counts.append(0)
        
    try:
        case.append(count[0].split(' ')[1])
    except IndexError:
        case.append('')

df['counts'] = counts
df['counts'] = [str(count).replace(',', '') for count in df['counts']]
df['counts'] = [str(count).replace('.', '') for count in df['counts']]
df['case'] = case

In [203]:
# Finalizing Locations

ph_loc = []
for loc in df['PH_Loc']:
    try:
        ph_loc.append(loc[0])
    except IndexError:
        ph_loc.append('')
df['Loc'] = ph_loc

In [204]:
# Fixing confirmed counts

count_fixer = []
for index, row in df.iterrows():
    if (row['status'] == 'confirmed') & (row['case'] != 'confirmed'):
        count_fixer.append(1)
    else:
        count_fixer.append(row['counts'])

df['counts'] = count_fixer

# Processing for CSV

In [205]:
df = df.reset_index(drop = True)
df.to_csv('abscbn_parsed.csv', index = False)

In [206]:
df = pd.read_csv('abscbn_parsed.csv')

In [207]:
df['source_id'] = 0

In [208]:
df = df.append(pd.read_csv('rappler_parsed.csv'))

In [209]:
df = df[df['counts'] != '1300\n\nConfirmed']
df = df[df['Loc'].isin(['cebu', 'aklan', 'tarlac','camiguin'])]


In [210]:
df['counts'] = df.counts.astype(int)

In [216]:
df = df.sort_values('date')

In [217]:
def parse(df):
    print(df.info())
    
    # Get min/max/mean values
    dfa = pd.pivot_table(df, values = 'counts', index=['date', 'Loc'], columns='status', aggfunc=[min, max, np.mean, stats.mode])
    
    # Remove multi-index
    dfa.columns = ["_".join(pair) for pair in dfa.columns]
    dfa = dfa.reset_index()
    
    # Replace 0 with np.nan to forward fill null values
    dfa = dfa.replace(0, np.nan)
    
    # Forward filling needs to be by area
    places = list(df['Loc'].unique())
    
    global dfb
    dfb = pd.DataFrame()
    for place in places:
        df_temp = dfa[dfa['Loc'] == place].fillna(method='ffill')
        dfb = dfb.append(df_temp)
    return dfb

In [218]:
res = parse(df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32 entries, 2 to 67
Data columns (total 13 columns):
LDA_Topics    32 non-null object
Loc           32 non-null object
PH_Loc        32 non-null object
author        32 non-null object
case          16 non-null object
category      32 non-null object
count_docs    32 non-null object
counts        32 non-null int32
date          32 non-null object
source_id     32 non-null int64
status        21 non-null object
text          32 non-null object
title         32 non-null object
dtypes: int32(1), int64(1), object(11)
memory usage: 3.4+ KB
None


In [219]:
res

,date,Loc,min_confirmed,min_suspected,max_confirmed,max_suspected,mean_confirmed,mean_suspected,mode_confirmed,mode_suspected
1,2020-01-23,cebu,NaN,NaN,NaN,555.0,NaN,185.0,NaN,"([0], [2])"
2,2020-01-24,cebu,NaN,14.0,NaN,14.0,NaN,14.0,NaN,"([14], [1])"
7,2020-01-30,cebu,NaN,14.0,NaN,14.0,NaN,14.0,NaN,"([0], [3])"
9,2020-01-31,cebu,1.0,100.0,1.0,100.0,1.0,100.0,"([1], [2])","([100], [1])"
10,2020-02-02,cebu,1.0,100.0,1.0,100.0,1.0,100.0,"([1], [1])","([100], [1])"
11,2020-02-04,cebu,1.0,100.0,1.0,100.0,1.0,100.0,"([1], [1])","([100], [1])"
12,2020-02-05,cebu,1.0,100.0,1.0,100.0,1.0,100.0,"([1], [2])","([100], [1])"
0,2020-01-23,aklan,NaN,3.0,NaN,3.0,NaN,3.0,NaN,"([3], [1])"
3,2020-01-25,aklan,NaN,80.0,NaN,80.0,NaN,80.0,NaN,"([80], [1])"
5,2020-01-28,aklan,NaN,11.0,NaN,11.0,NaN,11.0,NaN,"([11], [1])"


In [192]:
res = res[['date','Loc', 'min_suspected','min_confirmed']]

In [193]:
res = res.fillna(0)

In [194]:
res = res.sort_values('date')
res

,date,Loc,min_suspected,min_confirmed
2,2020-01-23,cebu,0.0,0.0
1,2020-01-23,aklan,3.0,0.0
4,2020-01-24,cebu,14.0,0.0
6,2020-01-25,aklan,80.0,0.0
8,2020-01-27,camiguin,106.0,0.0
10,2020-01-28,aklan,11.0,0.0
14,2020-01-30,cebu,14.0,0.0
13,2020-01-30,aklan,11.0,1.0
17,2020-01-31,cebu,100.0,1.0
16,2020-01-31,aklan,11.0,1.0


In [174]:
for Loc in res['Loc']:
    sus_holder = 0
    con_holder = 0
    for date in res['date']:
        if sum(res[res['date'] == date]['Loc'].str.contains(Loc)) > 0:
            
            sus_holder = res[(res['date'] == date) & (res['Loc'] == Loc)]['min_suspected'].iloc[0]
            con_holder = res[(res['date'] == date) & (res['Loc'] == Loc)]['min_confirmed'].iloc[0]
        else:
            add_row(res, [date, Loc, sus_holder, con_holder])
            

C:\Users\jrdaos\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [175]:
res = res.sort_values('date')

In [176]:
res

,date,Loc,min_suspected,min_confirmed
27,2020-01-23,aklan,3.0,0.0
28,2020-01-23,cebu,0.0,0.0
8,2020-01-23,tarlac,0.0,0.0
17,2020-01-23,camiguin,0.0,0.0
30,2020-01-24,cebu,14.0,0.0
7,2020-01-24,tarlac,0.0,0.0
16,2020-01-24,camiguin,0.0,0.0
25,2020-01-24,aklan,3.0,0.0
6,2020-01-25,tarlac,0.0,0.0
15,2020-01-25,camiguin,0.0,0.0


In [10]:
prov = gpd.read_file('prov_shp/province.shp')

In [177]:
df = pd.merge(res, prov, left_on = 'Loc', right_on = 'Pro_Name')

In [178]:
df.head()

,date,Loc,min_suspected,min_confirmed,Pro_Name,long,lat,geometry
0,2020-01-23,aklan,3.0,0.0,aklan,122.24802,11.609596,(POLYGON ((122.4047963630001 11.64533761000007...
1,2020-01-24,aklan,3.0,0.0,aklan,122.24802,11.609596,(POLYGON ((122.4047963630001 11.64533761000007...
2,2020-01-25,aklan,80.0,0.0,aklan,122.24802,11.609596,(POLYGON ((122.4047963630001 11.64533761000007...
3,2020-01-27,aklan,80.0,0.0,aklan,122.24802,11.609596,(POLYGON ((122.4047963630001 11.64533761000007...
4,2020-01-28,aklan,11.0,0.0,aklan,122.24802,11.609596,(POLYGON ((122.4047963630001 11.64533761000007...


In [179]:
df = df[['date','Loc','min_suspected','min_confirmed','long','lat']]
df.columns = (['Date','Location','Suspected','Confirmed','Longitude','Latitude'])

In [180]:
df['Date'] = [datetime.datetime.strptime(str(date), '%Y-%m-%d').strftime('%Y-%m-%dT%H:%M:%S.%f') for date in df['Date']]

In [181]:
df.sort_values('Date')

,Date,Location,Suspected,Confirmed,Longitude,Latitude
0,2020-01-23T00:00:00.000000,aklan,3.0,0.0,122.248020,11.609596
30,2020-01-23T00:00:00.000000,camiguin,0.0,0.0,124.717743,9.171998
10,2020-01-23T00:00:00.000000,cebu,0.0,0.0,123.754729,10.355054
20,2020-01-23T00:00:00.000000,tarlac,0.0,0.0,120.476206,15.478348
21,2020-01-24T00:00:00.000000,tarlac,0.0,0.0,120.476206,15.478348
1,2020-01-24T00:00:00.000000,aklan,3.0,0.0,122.248020,11.609596
31,2020-01-24T00:00:00.000000,camiguin,0.0,0.0,124.717743,9.171998
11,2020-01-24T00:00:00.000000,cebu,14.0,0.0,123.754729,10.355054
2,2020-01-25T00:00:00.000000,aklan,80.0,0.0,122.248020,11.609596
12,2020-01-25T00:00:00.000000,cebu,14.0,0.0,123.754729,10.355054


In [182]:
df.to_csv('ncov_parsed.csv', index = False)